In [1]:
# Wallet Risk Scoring From Scratch
# Siddharth Hiriyan - Risk Modeling for Compound Wallets

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


In [2]:

# STEP 1: Load Wallet Data
wallet_df = pd.read_csv('/content/Wallet id - Sheet1.csv')  # <-- Adjust path in Colab


In [3]:

# Rename column for consistency
wallet_df.columns = ['wallet_id']



In [4]:
# STEP 2: Simulate Feature Extraction (Mocked for now)
# Replace this section with actual API calls to Compound later

np.random.seed(42)  # for reproducibility
wallet_df['total_borrows'] = np.random.uniform(100, 50000, size=len(wallet_df))
wallet_df['total_repaid'] = wallet_df['total_borrows'] * np.random.uniform(0.6, 1.2, size=len(wallet_df))
wallet_df['collateral_ratio'] = np.random.uniform(0.5, 3.0, size=len(wallet_df))
wallet_df['liquidation_events'] = np.random.randint(0, 5, size=len(wallet_df))
wallet_df['tx_frequency'] = np.random.randint(5, 100, size=len(wallet_df))  # interactions with Compound



In [5]:
# STEP 3: Normalize Features
features = ['total_borrows', 'total_repaid', 'collateral_ratio', 'liquidation_events', 'tx_frequency']
scaler = MinMaxScaler()
wallet_df[[f + '_norm' for f in features]] = scaler.fit_transform(wallet_df[features])



In [6]:
# STEP 4: Risk Scoring (lower score = higher risk)
# Heuristic:
# Higher liquidation = bad (↑ risk)
# Lower collateral ratio = bad (↑ risk)
# Lower repayment % = bad
# Low tx frequency = bad

wallet_df['repay_ratio_norm'] = wallet_df['total_repaid_norm'] / (wallet_df['total_borrows_norm'] + 1e-6)

# Weighting strategy (tweak as needed)
w_liq = 0.4
w_col = 0.25
w_repay = 0.2
w_freq = 0.15

wallet_df['risk_score_raw'] = (
    w_liq * wallet_df['liquidation_events_norm'] +
    w_col * (1 - wallet_df['collateral_ratio_norm']) +
    w_repay * (1 - wallet_df['repay_ratio_norm']) +
    w_freq * (1 - wallet_df['tx_frequency_norm'])
)



In [7]:
# STEP 5: Convert to Final Score (0 = worst, 1000 = best)
wallet_df['score'] = (1 - wallet_df['risk_score_raw']) * 1000
wallet_df['score'] = wallet_df['score'].clip(0, 1000).astype(int)



In [8]:
# STEP 6: Save to CSV
output_df = wallet_df[['wallet_id', 'score']]
output_df.to_csv('/content/wallet_risk_scores.csv', index=False)



In [9]:
# Display sample output
output_df.head()

,wallet_id,score
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,416
1,0x06b51c6882b27cb05e712185531c1f74996dd988,527
2,0x0795732aacc448030ef374374eaae57d2965c16c,445
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,606
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,364
